In [30]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [31]:
csv_file_path = '/home/lucas/UFOP/ple_2020/analise_midias_sociais/final-work/data/all_pp_posts_haddad_replaced_to_abbreviation.csv'
df_candidate = pd.read_csv(csv_file_path,encoding = 'utf-8')

In [32]:
df_candidate.head(15)

,created_time,id,message_max_processed,m_m_p_replaced_to_abbreviation,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
0,2020-02-29T11:23:27+0000,904277726319518_2878974912183113,bolsonavirus nada erroneo considerar bolsonaro...,bolsonavirus nada erroneo considerar jb aciden...,818,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,4996,209,19,40,58,636,True
1,2020-02-29T09:21:22+0000,904277726319518_2878783568868914,bolsonavirus nada erroneo considerar bolsonaro...,bolsonavirus nada erroneo considerar jb raio c...,477,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,2602,84,14,26,19,130,True
2,2020-02-28T00:14:25+0000,904277726319518_2875616525852285,cristiano zanin advogado lula fernando haddad ...,cristiano zanin advogado lula fh discutem novi...,540,mobile_status_update,https://scontent-gru2-2.xx.fbcdn.net/v/t15.525...,3068,120,23,4,20,140,True
3,2020-02-26T12:12:56+0000,904277726319518_2872245456189392,disney pao ha ano governo bolsonaro ja sabia f...,disney pao ha ano governo jb ja sabia falta di...,1068,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,3148,48,24,1306,616,8,True
4,2020-02-26T00:58:59+0000,904277726319518_2871255852955019,bolsonaro tudo indica cometeu crime responsabi...,jb tudo indica cometeu crime responsabilidade ...,1369,mobile_status_update,None,21619,524,79,68,77,1923,True
5,2020-02-24T19:51:01+0000,904277726319518_2868531243227480,impacto fundeb criamos qualidade educacao nord...,impacto fundeb criamos qualidade educacao nord...,3249,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,10571,86,45,9,2,2635,True
6,2020-02-22T13:03:28+0000,904277726319518_2863199017094036,cronica semana nao podemos descuidar dois prob...,cronica semana nao podemos descuidar dois prob...,528,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,2718,80,31,130,34,31,True
7,2020-02-22T00:18:55+0000,904277726319518_2862087573871847,bem falaram carnaval rua sp nao ia rolar trist...,bem falaram carnaval rua sp nao ia rolar trist...,1289,added_photos,https://scontent-gru1-1.xx.fbcdn.net/v/t1.0-9/...,29857,1003,132,25,14,12585,True
8,2020-02-20T11:28:49+0000,904277726319518_2858610960886175,ha clara tentativa desestabilizar melhores gov...,ha clara tentativa desestabilizar melhores gov...,588,mobile_status_update,None,9914,223,12,19,7,820,True
9,2020-02-18T20:38:19+0000,904277726319518_2855332637880674,brasilia discutindo oposicao brasil justo demo...,brasilia discutindo oposicao brasil justo demo...,541,added_photos,https://scontent-gru2-1.xx.fbcdn.net/v/t1.0-9/...,9555,369,32,3,20,2114,True


In [33]:
n_total_posts = len(df_candidate)
print('número total de posts: {0}'.format(str(n_total_posts)))

número total de posts: 1880


In [34]:
n_posts_with_textual_messages = df_candidate[df_candidate.has_textual_message == True].shape[0]
n_posts_without_textual_messages = df_candidate[df_candidate.has_textual_message == False].shape[0]

print('número de posts com mensagens textuais: {0}'.format(str(n_posts_with_textual_messages)))
print('número de posts sem mensagens textuais: {0}'.format(str(n_posts_without_textual_messages)))

número de posts com mensagens textuais: 1799
número de posts sem mensagens textuais: 81


In [35]:
# Devemos remover esses posts sem menssagem antes de continuar a nossa análise?
# renomear depois o df caso sim

#filter_mask  = df_candidate['has_textual_message'] == True
#df_candidate = df_candidate[filter_mask]
#print('número de posts do df desconsiderando os posts sem menssagens textuais: {0}'.format(str(len(df_candidate))))

In [36]:
# separando o df de treinamento (30%) e o df de testes(70%) de maneira 'embaralhada'
df_training, df_test = train_test_split(df_candidate,train_size=0.7,test_size=0.3,shuffle=False)

In [37]:
print('número de posts do conjunto de treinamento (70%): {0}'.format(str(len(df_training))))
df_training.head(5)

número de posts do conjunto de treinamento (70%): 1316


,created_time,id,message_max_processed,m_m_p_replaced_to_abbreviation,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
0,2020-02-29T11:23:27+0000,904277726319518_2878974912183113,bolsonavirus nada erroneo considerar bolsonaro...,bolsonavirus nada erroneo considerar jb aciden...,818,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,4996,209,19,40,58,636,True
1,2020-02-29T09:21:22+0000,904277726319518_2878783568868914,bolsonavirus nada erroneo considerar bolsonaro...,bolsonavirus nada erroneo considerar jb raio c...,477,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,2602,84,14,26,19,130,True
2,2020-02-28T00:14:25+0000,904277726319518_2875616525852285,cristiano zanin advogado lula fernando haddad ...,cristiano zanin advogado lula fh discutem novi...,540,mobile_status_update,https://scontent-gru2-2.xx.fbcdn.net/v/t15.525...,3068,120,23,4,20,140,True
3,2020-02-26T12:12:56+0000,904277726319518_2872245456189392,disney pao ha ano governo bolsonaro ja sabia f...,disney pao ha ano governo jb ja sabia falta di...,1068,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,3148,48,24,1306,616,8,True
4,2020-02-26T00:58:59+0000,904277726319518_2871255852955019,bolsonaro tudo indica cometeu crime responsabi...,jb tudo indica cometeu crime responsabilidade ...,1369,mobile_status_update,None,21619,524,79,68,77,1923,True


In [38]:
print('número de posts do conjunto de teste (30%): {0}'.format(str(len(df_test))))
df_test.head(5)

número de posts do conjunto de teste (30%): 564


,created_time,id,message_max_processed,m_m_p_replaced_to_abbreviation,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
1316,2017-08-05T00:22:06+0000,904277726319518_1497518800328738,continuacao aula inaugural instituto dom jose ...,continuacao aula inaugural instituto dom jose ...,67,added_video,https://scontent-gru2-1.xx.fbcdn.net/v/t15.525...,534,3,1,0,1,94,True
1317,2017-08-04T23:28:13+0000,904277726319518_1497463580334260,chapeco sc instituto dom jose gomes idjg aula ...,chapeco sc instituto dom jose gomes idjg aula ...,208,added_video,https://scontent-gru2-2.xx.fbcdn.net/v/t15.525...,1160,9,6,2,1,213,True
1318,2017-08-01T20:39:00+0000,904277726319518_1494003407346944,reformulamos completamente enem acabar vestibu...,reformulamos completamente enem acabar vestibu...,1804,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,8262,6,226,3,4,1155,True
1319,2017-08-01T13:37:02+0000,904277726319518_1493772520703366,crime urbanisticoambiental desculpem textao as...,crime urbanisticoambiental desculpem textao as...,1203,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,3682,6,47,1191,908,16,True
1320,2017-08-01T01:00:37+0000,904277726319518_1493347450745873,plano diretor premiado onuhabitat visitado sit...,plano diretor premiado onuhabitat visitado sit...,457,shared_story,https://external-gru1-1.xx.fbcdn.net/safe_imag...,3971,0,26,2,0,558,True


In [39]:
#============================================================================================
# realizando o tf idf no df['pre_processed_message'] de treinamento
#============================================================================================

# tf idf é uma medida estatística que tem o intuito de indicar a importância de uma palavra 
# de um documento em relação a uma coleção de documentos ou em um corpus linguístico.

# instanciando o objeto para fazer o tf-idf
tfidf_vectorizer = TfidfVectorizer(analyzer='word')

# converte os documentos em matrizes
tfidf_matrix = tfidf_vectorizer.fit_transform(df_training['m_m_p_replaced_to_abbreviation'])

# obtendo as palavras (tokens)
tfidf_tokens = tfidf_vectorizer.get_feature_names()

# gerando o tf-idf
# colunas são as palavras processadas
# linhas são os documentos (post 1, post 2,post 3,post 4)
df_tfidfvect = pd.DataFrame(data = tfidf_matrix.toarray(),columns = tfidf_tokens)


print("\nTF-IDF\n")
df_tfidfvect.head(10)


TF-IDF



,abaixar,abaixofakenews,abandonar,abandonara,abastecer,abc,abel,aberracao,aberta,abertas,...,zanin,zapatero,zeladoria,zerada,zero,zika,zimmer,zona,zumbi,zumbis
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.176234,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.215954,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# Aplicando uma máscara para identificar o tf-idf de certas palavras

filter_mask  = df_tfidfvect['abaixo'] != 0.0
df_tfidf_token_mask = df_tfidfvect[filter_mask]

print(df_tfidf_token_mask)

KeyError: 'abaixo'

In [41]:
#============================================================================================
# Implementacao do lda
#============================================================================================

In [42]:
cv = CountVectorizer()
df = cv.fit_transform(df_training['m_m_p_replaced_to_abbreviation'])

In [43]:
model_lda = LatentDirichletAllocation(n_components = 30,random_state = 0)
model_lda.fit(df)

LatentDirichletAllocation(n_components=30, random_state=0)

In [44]:
# objetivo: tentar encontrar um numero bom de tópicos a fim de conseguirmos destinguir as
# categorias 

for index, topic in enumerate(model_lda.components_):
    print(f'Top 10 words for #{index}')
    main_words_for_topic = [cv.get_feature_names()[i] for i in topic.argsort()[-10:]]
    print(main_words_for_topic)
    message = ' '
    message = message.join(main_words_for_topic)
    print(message)
    print('\n')
    

Top 10 words for #0
['bracos', 'hospital', 'obra', 'brasil', 'parte', 'poder', 'rede', 'sobre', 'dei', 'entrevista']
bracos hospital obra brasil parte poder rede sobre dei entrevista


Top 10 words for #1
['like', 'these', 'your', 'all', 'my', 'in', 'the', 'here', 'and', 'still']
like these your all my in the here and still


Top 10 words for #2
['democracia', 'hoje', 'defesa', 'contra', 'todos', 'atos', 'sobre', 'brasil', 'jb', 'nao']
democracia hoje defesa contra todos atos sobre brasil jb nao


Top 10 words for #3
['coluna', 'republica', 'presidencia', 'lula', 'direito', 'eleicoes', 'jb', 'semana', 'agora', 'sobre']
coluna republica presidencia lula direito eleicoes jb semana agora sobre


Top 10 words for #4
['porto', 'tv', 'democracia', 'coligacao', 'presidente', 'eleitoral', 'povo', 'acompanhe', 'assista', 'programa']
porto tv democracia coligacao presidente eleitoral povo acompanhe assista programa


Top 10 words for #5
['governador', 'janeiro', 'pessoa', 'estado', 'lula', 'joao